In [93]:
import pandas as pd

In [94]:
inv = pd.read_csv("../data/opendata/samples/inventions_sample.csv")
mod = pd.read_csv("../data/opendata/samples/models_sample.csv")
des = pd.read_csv("../data/opendata/samples/designs_sample.csv")

In [95]:
inv_res = pd.read_excel("inv_test_result.xlsx", index_col=0)
mod_res = pd.read_excel("mod_test_result.xlsx", index_col=0)
des_res = pd.read_excel("des_test_result.xlsx", index_col=0)

In [96]:
mod_res.head(2)

,patent_number,cor_address,name,tax_number,name_from_patent,individual,country,has_tn
0,120365,"660133, г.Красноярск, ул. Авиаторов, 1, стр.1,...","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""АРНИКА""",2.460019e+09,"Общество с ограниченной ответственностью ""Арника""",False,RU,True
1,32373,"198328, Санкт-Петербург, пр-т маршала Захарова...",БЕЛЯЕВ АЛЕКСАНДР ГЕННАДЬЕВИЧ,7.807084e+11,Беляев Александр Геннадьевич,True,RU,True


In [97]:
def prepare_part(d, d_with_res, kind):
    df = d_with_res.merge(
        d, how="left", left_on="patent_number", right_on="registration number"
    )
    df["reg_number"] = df["patent_number"]
    df["reg_date"] = pd.to_datetime(df["registration date"], format="%Y%m%d")
    df["appl_date"] = pd.to_datetime(df["application date"], format="%Y%m%d")
    df["author_raw"] = df["authors"].str[:100]
    df["owner_raw"] = df["patent holders"].str[:100]
    df["address"] = df["cor_address"]
    df["full_name"] = df["name"].str[:100]
    df["name"] = df[[c for c in df.columns if " name" in c][0]]
    df["class"] = 1
    df["subclass"] = 1
    df["kind"] = kind    
    
    return df[[
        "reg_number", "reg_date", "appl_date",
        "author_raw", "owner_raw", "address",
        "name", "actual", "class", "subclass", "kind",
        "individual", "full_name", "tax_number"
    ]]

In [98]:
joint = pd.concat((
    prepare_part(inv, inv_res, 1),
    prepare_part(mod, mod_res, 2),
    prepare_part(des, des_res, 3),
))

In [99]:
joint.tail(2)

,reg_number,reg_date,appl_date,author_raw,owner_raw,address,name,actual,class,subclass,kind,individual,full_name,tax_number
1065,78299,2011-05-16,2010-03-22,Желез Наталья Ильинична (RU)\r\nЖелез Сергей Б...,Желез Наталья Ильинична (RU),"440058, г. Пенза, ул. Тепличная, 8, кв. 205, Н...",ДИВАН УГЛОВОЙ СЕКЦИОННЫЙ,False,1,1,3,True,ЖЕЛЕЗ НАТАЛЬЯ ИЛЬИНИЧНА,5.834076e+11
1066,59842,2006-08-16,2005-03-21,Черногорцев Дмитрий Викторович (RU)\r\nБурнашо...,"Общество с ограниченной ответственностью ""Бабк...",NaN,УПАКОВКА ДЛЯ СЕМЕЧЕК (два варианта),False,1,1,3,False,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ТОРГ...",7.725776e+09


In [100]:
reg_numbers = joint["reg_number"].unique()
reg_number_ids = pd.DataFrame(dict(reg_number=reg_numbers, patent_id=range(1, len(reg_numbers) + 1)))
reg_number_ids.head(2)

,reg_number,patent_id
0,2137261,1
1,2631279,2


In [101]:
tax_numbers = joint["tax_number"].unique()
tax_number_ids = pd.DataFrame(dict(tax_number=tax_numbers, person_id=range(1, len(tax_numbers) + 1)))
tax_number_ids.head(2)

,tax_number,person_id
0,5.034093e+11,1
1,7.710140e+09,2


In [102]:
full = joint.merge(reg_number_ids).merge(tax_number_ids)
full.head(2)

,reg_number,reg_date,appl_date,author_raw,owner_raw,address,name,actual,class,subclass,kind,individual,full_name,tax_number,patent_id,person_id
0,2137261,1999-09-10,1998-08-04,Демидов Юрий Михайлович,Демидов Юрий \n\nМихайлович,"127560, Москва, ул.Коненкова, 5, кв.16, Демидо...",АКТИВНЫЙ МАТЕРИАЛ ЭЛЕКТРОДА ХИМИЧЕСКОГО ИСТОЧН...,False,1,1,1,False,ДЕМИДОВ ЮРИЙ ПЕТРОВИЧ,5.034093e+11,1,1
1,2137261,1999-09-10,1998-08-04,Демидов Юрий Михайлович,Демидов Юрий \n\nМихайлович,"127560, Москва, ул.Коненкова, 5, кв.16, Демидо...",АКТИВНЫЙ МАТЕРИАЛ ЭЛЕКТРОДА ХИМИЧЕСКОГО ИСТОЧН...,False,1,1,1,False,XL,7.710140e+09,1,2


In [103]:
patents = full[[
    "reg_number", "reg_date", "appl_date",
    "author_raw", "owner_raw", "address",
    "name", "actual", "class", "subclass", "kind",
    "patent_id"
]].drop_duplicates("patent_id")
patents.rename(columns={"patent_id": "id"}, inplace=True)
patents.head(2)

,reg_number,reg_date,appl_date,author_raw,owner_raw,address,name,actual,class,subclass,kind,id
0,2137261,1999-09-10,1998-08-04,Демидов Юрий Михайлович,Демидов Юрий \n\nМихайлович,"127560, Москва, ул.Коненкова, 5, кв.16, Демидо...",АКТИВНЫЙ МАТЕРИАЛ ЭЛЕКТРОДА ХИМИЧЕСКОГО ИСТОЧН...,False,1,1,1,1
3,2631279,2017-09-20,2016-03-18,Кочетов Олег Савельевич (RU),Кочетов Олег Савельевич (RU),"141191, Московская обл., г. Фрязино, ул. Горьк...",ШИРОКОФАКЕЛЬНАЯ ЦЕНТРОБЕЖНАЯ ФОРСУНКА,True,1,1,1,2


In [104]:
persons = full[[
    "individual", "tax_number", "full_name", "person_id"
]].drop_duplicates("person_id")
persons.rename(columns={"person_id": "id", "full_name": "name", "individual": "kind"}, inplace=True)
persons["kind"] = persons["kind"].astype(int)
persons["tax_number"] = persons["tax_number"].astype(str).str[:12]
persons.head(2)

,kind,tax_number,name,id
0,0,503409279199,ДЕМИДОВ ЮРИЙ ПЕТРОВИЧ,1
1,0,7710140407.0,XL,2


In [105]:
ownership = full[["patent_id", "person_id"]]
ownership.head(2)

,patent_id,person_id
0,1,1
1,1,2


In [106]:
patents.to_csv("patents-demo.csv", index=False)
persons.to_csv("persons-demo.csv", index=False)
ownership.to_csv("ownership-demo.csv", index=False)